### notebook for parsing netlist XML files

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
class Component:
    def __init__(self, ref, value, mask=None):
        self.ref = ref
        self.value = value
        self.mask = mask # mask could be filled after image processing stage

    # when i call print(Component), i want to see the ref and value
    def __str__(self):
        return f'{self.ref} {self.value}'

class Net:
    def __init__(self, code, name):
        self.code = code
        self.name = name # name is an alternative custom name for a net (like VCC, GND, AMP_INPUT, etc.)
        self.nodes = [] # each node should have a reference and a pin number

    def __str__(self):
        return f'{self.code} {self.name}'


class Node:
    def __init__(self, ref, pin):
        self.ref = ref
        self.pin = pin

    def __str__(self):
        return f'{self.ref} {self.pin}'

# ideally, at the end of the breadboard image processing, we'll end up with a list of Component objects and a list of Net objects

In [39]:
import re

path_to_netlist = "../data/emg-analog-front-end.net"

components = [] # array of Component objects
nets = []

# populate components array based on the netlist file
with open(path_to_netlist, "r") as f:
    lines = f.readlines()
    current_component = None
    for i in range(len(lines)):
        line = lines[i]
        if "(comp (" in line:
            ref_str = lines[i]
            value_str = lines[i+1]
            ref = ref_str[ref_str.find('"')+1:ref_str.rfind('"')]
            value = value_str[value_str.find('"')+1:value_str.rfind('"')]
            component = Component(ref, value)
            components.append(component)
            i += 10 # component has 10 parameters in a netlist file

        if "(net (" in line:
            # "(net (code "1") (name "-VDD")"
            net_code, net_name = None, None

            code_match = re.search(r'\(code "([^"]+)"\)', line)
            if code_match:
                net_code = code_match.group(1)
            else:
                net_code = None

            name_match = re.search(r'\(name "([^"]+)"\)', line)
            if name_match:
                net_name = name_match.group(1)
            else:
                net_name = None

            net = Net(net_code, net_name)
            i += 1
            line = lines[i] # "(node (ref "C11") (pin "1") (pintype "passive"))"
            
            nodes = []
            while "node" in line:
                # "(node (ref "C11") (pin "1") (pintype "passive"))"
                node_ref, node_pin = None, None

                ref_match = re.search(r'\(ref "([^"]+)"\)', line)
                if ref_match:
                    node_ref = ref_match.group(1)
                else:
                    node_ref = None

                pin_match = re.search(r'\(pin "([^"]+)"\)', line)
                if pin_match:
                    node_pin = pin_match.group(1)
                else:
                    node_pin = None

                node = Node(ref, pin)
                nodes.append(node)
                i += 1
                if i >= len(lines):
                    break
                line = lines[i]
            net.nodes = nodes
            print(net, len(net.nodes))

        nets.append(net)

    

1 -VDD 10
4 /HPF_IN 5
5 /IA_IN+ 2
6 /IA_IN- 2
8 AMP_IN 3
9 AOUT 4
10 AOUT1 2
11 E+ 3
12 E- 3
13 GND 19
14 HPF_OUT 6
15 LPF_IN 3
16 LPF_OUT 4
17 NOTCH_IN 4
18 NOTCH_OUT 3
19 Net-(C13-Pad2) 3
20 Net-(C14-Pad2) 3
21 Net-(C16-Pad1) 2
22 Net-(C16-Pad2) 2
23 Net-(H6-Pad1) 2
24 Net-(H7-Pad1) 2
25 Net-(H8-Pad1) 2
26 Net-(R5-Pad1) 2
27 Net-(R5-Pad2) 2
28 Net-(R7-Pad2) 2
29 Net-(R7-Pad3) 3
30 Net-(R14-Pad1) 3
31 Net-(R14-Pad2) 3
32 SK00 3
33 SK01 3
34 VDD 11


[]